In [295]:
import qiskit
from qiskit import QuantumCircuit, ClassicalRegister
from qiskit.quantum_info import Statevector
from qiskit import BasicAer

In [296]:
def create_large_entangled(primed=None):
    qr = qiskit.QuantumRegister(6)
    #cr = qiskit.ClassicalRegister(6)
    c = qiskit.QuantumCircuit(qr)#, cr)
    
    if primed is not None:
        prime_circuit(c, qr, primed)
    create_bell_pair(c, qr[0], qr[1])
    for i in range(2, 6):
        c.h(qr[i])
    
    import random
    current_loc_1 = 0
    current_loc_2 = 1
    for i in range(10):
        r = random.randint(0, 5)
        while r == current_loc_1:
            r = random.randint(0, 5)
        if r == current_loc_2:
            current_loc_2 = current_loc_1
        c.swap(qr[current_loc_1], qr[r])
        current_loc_1 = r
    for i in range(10):
        r = random.randint(0, 5)
        while r == current_loc_2:
            r = random.randint(0, 5)
        if r == current_loc_1:
            current_loc_1 = current_loc_2
        c.swap(qr[current_loc_2], qr[r])
        current_loc_2 = r
    return c, current_loc_1, current_loc_2

In [297]:
def create_bell_pair(circuit, qubit_1, qubit_2):
    circuit.h(qubit_1)
    circuit.cnot(qubit_1, qubit_2)

In [298]:
def prime_circuit_initialize(circuit, qubit_list, bitstring):
    n = len(qubit_list)
    circuit.initialize(Statevector.from_label(bitstring).data, range(n))

    return circuit

In [299]:
def prime_circuit(circuit, qubit_list, bitstring):
    n = len(qubit_list)
#    cr = ClassicalRegister(n, 'c')
#    circuit.add_register(cr)
#    circuit.measure([0,1,2,3,4,5], [0,1,2,3,4,5])
#    qasm_string = cr.qasm()
#    print("qasm_string:", qasm_string)
    for i in range(n):
        if bitstring[i] == 1:
            circuit.x(i)            
            
    return circuit

In [300]:
c, q1, q2 = create_large_entangled(primed="101010")
#print(c)
#print(q1, q2)

In [301]:
c.measure_all()
counts = qiskit.execute(c, backend=BasicAer.get_backend('qasm_simulator')).result().get_counts()
counts

{'000010': 30,
 '000110': 31,
 '111111': 27,
 '100100': 34,
 '011101': 38,
 '000100': 28,
 '111001': 33,
 '101101': 39,
 '001011': 36,
 '110000': 36,
 '110110': 28,
 '101001': 25,
 '010000': 29,
 '100000': 42,
 '001001': 41,
 '111101': 38,
 '100110': 29,
 '001101': 37,
 '011111': 32,
 '010110': 29,
 '011011': 25,
 '111011': 33,
 '101111': 32,
 '110100': 36,
 '010010': 27,
 '001111': 35,
 '000000': 30,
 '110010': 23,
 '011001': 25,
 '010100': 34,
 '100010': 35,
 '101011': 27}

In [302]:
def find_entangled_pair_general(counts):
    
    # Pair must be 00 or 11
    if "000000" in counts.keys():
        print("00 or 11")
    else:
        print("01 or 10")
  
    bitstring_length = len(list(counts.keys())[0])
    not_entangled_qubits = set()
    for bitstring in counts:
        # this qubit is not entangled
        if bitstring.count("1") == 1:
            not_entangled_qubits.add(bitstring_length - 1 - bitstring.find("1"))            
    entangled_qubits = list()
    for i in range(bitstring_length):        
        if i not in not_entangled_qubits:
            entangled_qubits.append(i)    
    if len(entangled_qubits) == 2:
        qubit_1 = entangled_qubits[0]
        qubit_2 = entangled_qubits[1]
    else:
        not_entangled_qubits_list = list(not_entangled_qubits)
        qubit_1 = not_entangled_qubits_list[0] 
        qubit_2 = not_entangled_qubits_list[1]
        
    print("size entangled_qubits:", len(entangled_qubits), entangled_qubits)    
        
    return qubit_1, qubit_2

In [303]:
res = find_entangled_pair_general(counts)
print(q1, q2, res[0], res[1])

00 or 11
size entangled_qubits: 2 [0, 3]
0 3 0 3
